In [1]:
import math
from multiprocessing import Pool, Manager
import os
from copy import copy
from os.path import join

import numpy as np
import pandas as pd
import scipy.ndimage
from scipy.spatial import distance
import skimage.color
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
from tqdm.autonotebook import tqdm

from visual_words import *
from visual_recog import *

<ipython-input-1-449f49e3cfb1>:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
from opts import get_options

opts = get_options()


In [3]:
# utils

def get_num_CPU():
    '''
    Counts the number of CPUs available in the machine.
    '''
    return multiprocessing.cpu_count()


def display_filter_responses(img_name, opts, response_maps):
    '''
    Visualizes the filter response maps.

    [input]
    * response_maps: a numpy.ndarray of shape (H,W,3F)
    '''

    mode          = opts.mode
    n_scale       = len(opts.filter_scales)
    num_filters   = opts.num_filters
    num_channels  = opts.num_channels
    plt.figure(figsize=(15,15))
    for i in range(n_scale * num_filters):
        plt.subplot(n_scale, num_filters, i + 1)
        resp = response_maps[:, :, i * num_channels:i * num_channels + num_channels]
        resp_min = resp.min(axis=(0, 1), keepdims=True)
        resp_max = resp.max(axis=(0, 1), keepdims=True)
        resp = (resp - resp_min) / (resp_max - resp_min)
        plt.imshow(resp)
        plt.axis("off")

    plt.subplots_adjust(left=0.05, right=0.95, top=0.95,
                        bottom=0.05, wspace=0.05, hspace=0.05)
    img_name = "_".join([mode, img_name.split("/")[0],img_name.split("/")[1]])
    plt.savefig(os.path.join(opts.res_dir, img_name), pad_inches=0)
    plt.show()

def visualize_wordmap(original_image, wordmap, out_path=None):
    fig = plt.figure(figsize=(12.8, 4.8))
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(original_image)
    plt.axis("off")
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(wordmap)
    plt.axis("off")
    plt.show()
    if out_path:
        plt.savefig(out_path, pad_inches=0)


In [ ]:
"""
Apply all 4 filters at least 3 scales on aquarium/sun aztvjgubyrgvirup.jpg,
 visualize the responses
"""

opts.filter_scales = [1, 2, 4, 8, 12]
opts.K=10
opts.alpha=25
img_name = 'aquarium/sun_aztvjgubyrgvirup.jpg'
img_path = join(opts.data_dir, img_name)
#normalize the input image
img = plt.imread(img_path) / 255.
filter_responses = extract_filter_responses(opts, img)
display_filter_responses(img_name, opts, filter_responses)

In [ ]:
"""
Create a dictionary of visual words from the filter responses using k-means.
After applying k-means, similar filter responses will be represented by the same visual word. 

Given a dataset, generate a word dictionary.


"""
opts = get_options()

n_cpu = get_num_CPU()
opts.filter_scales = [1, 2]

dict_name = "dictionary_K120alpha60"
compute_dictionary(opts, dict_name, n_worker=5)

dict_file_name = dict_name + ".npy"
dictionary = np.load(join(opts.res_dir, dict_file_name))

In [ ]:
img_path = join(opts.data_dir, 'kitchen/sun_aasmevtpkslccptd.jpg')
img = plt.imread(img_path) / 255.
wordmap = get_visual_words(opts, img, dictionary)
visualize_wordmap(img, wordmap)

img_path = join(opts.data_dir, 'highway/sun_ailjxpgyepocjdos.jpg')
img = plt.imread(img_path) / 255.
wordmap = get_visual_words(opts, img, dictionary)
visualize_wordmap(img, wordmap)

img_path = join(opts.data_dir, 'laundromat/sun_aabvooxzwmzzvwds.jpg')
img = plt.imread(img_path) / 255.
wordmap = get_visual_words(opts, img, dictionary)
visualize_wordmap(img, wordmap)


In [ ]:
img_path = join(opts.data_dir, 'laundromat/sun_aabvooxzwmzzvwds.jpg')
img = plt.imread(img_path) / 255.
wordmap = get_visual_words(opts, img, dictionary)

hist = get_feature_from_wordmap(opts, wordmap, dictionary)
plt.bar(range(len(dictionary)), hist)
plt.title('Visual word histogram for laundromat/sun_aabvooxzwmzzvwds with K=50')
hist.shape

In [4]:
dict_name = "dictionary_K120alpha60"
opts.filter_scales = [1,2]
opts.K = 120
opts.alpha = 60
opts.L = 4
# compute_dictionary(opts, dict_name, n_worker=5)
# build_recognition_system(opts, dict_name+".npy", n_worker=5)
conf, accuracy = evaluate_recognition_system(opts, distance_to_set, n_worker=1)

print("Accuracy:", accuracy)
classes = [
    "aquarium", "desert", "highway", "kitchen",
    "laundromat", "park", "waterfall", "windmill",
]
df = pd.DataFrame(conf, columns=classes)
df.insert(0, "", classes)
df

Accuracy: 0.6325


,,aquarium,desert,highway,kitchen,laundromat,park,waterfall,windmill
0,aquarium,39.0,1.0,0.0,0.0,2.0,1.0,4.0,3.0
1,desert,1.0,33.0,1.0,1.0,5.0,3.0,1.0,5.0
2,highway,1.0,4.0,29.0,1.0,1.0,2.0,3.0,9.0
3,kitchen,2.0,3.0,1.0,29.0,11.0,1.0,2.0,1.0
4,laundromat,1.0,1.0,3.0,12.0,25.0,3.0,4.0,1.0
5,park,2.0,0.0,5.0,1.0,0.0,37.0,4.0,1.0
6,waterfall,2.0,0.0,4.0,2.0,3.0,7.0,30.0,2.0
7,windmill,0.0,2.0,10.0,0.0,2.0,4.0,1.0,31.0


In [ ]:
#from 3.1, the best performing parameters
opts.filter_scales = [1,2]
opts.K = 120
opts.alpha = 60
opts.L = 4
classifier = svm_classifier(opts)
conf, accuracy = evaluate_rec_svm(opts, classifier)
print("Accuracy: for L = 4 and K ",opts.K," is ", accuracy)